In [5]:
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

bitcoin_market_info = pd.read_csv("combined.csv")
bitcoin_market_info.head()
model_data = bitcoin_market_info

split_date = '2017-06-01'
training_set, test_set = model_data[model_data['date']<split_date], model_data[model_data['date']>=split_date]
training_set = training_set.drop('date', 1)
training_set = training_set.drop('day', 1)
test_set = test_set.drop('date', 1)
test_set = test_set.drop('day', 1)

In [6]:
print(training_set)

            price    difficulty  recipients  num_tranc
0        0.000000  1.000000e+00           0       14.0
1        0.000000  1.000000e+00           0      106.0
2        0.000000  1.000000e+00           0      116.0
3        0.000000  1.000000e+00           0      136.0
4        0.000000  1.000000e+00           0      109.0
5        0.000000  1.000000e+00           1      120.0
6        0.000000  1.000000e+00           0      115.0
7        0.000000  1.000000e+00           1       68.0
8        0.000000  1.000000e+00           0      195.0
9        0.000000  1.000000e+00           0      105.0
10       0.000000  1.000000e+00           1      122.0
11       0.000000  1.000000e+00           1      129.0
12       0.000000  1.000000e+00           1      127.0
13       0.000000  1.000000e+00           0      126.0
14       0.000000  1.000000e+00           0      129.0
15       0.000000  1.000000e+00           1      135.0
16       0.000000  1.000000e+00           0      127.0
17       0

In [7]:
#print(inputs)
LSTM_training_outputs = preprocessing.scale(training_set["price"])
LSTM_training_outputs  = LSTM_training_outputs[10:]

In [8]:
def get_input(inputs):
    result = []
    for i in range(0,len(inputs)-10):
        input_ = []
        #print(input_)
        for p in range(10):
            index = i + 1 + p
            input_.append(inputs[index])
        input_ = np.array(input_)
        result.append(input_) 
    return np.array(result)

In [9]:
window_len = 10
norm_cols = ["difficulty","price","recipients","num_tranc"]
# 1 remove difficulty
# 2 remove price
# 3 remove recipients
# 4 remove num_tranc

from sklearn import preprocessing

inputs_1 = preprocessing.scale(training_set.drop("difficulty",1))
inputs_1 = get_input(inputs_1.tolist())
inputs_2 = preprocessing.scale(training_set.drop("price",1))
inputs_2 = get_input(inputs_2.tolist())
inputs_3 = preprocessing.scale(training_set.drop("recipients",1))
inputs_3 = get_input(inputs_3.tolist())
inputs_4 = preprocessing.scale(training_set.drop("num_tranc",1))
inputs_4 = get_input(inputs_4.tolist())
inputs_5 = preprocessing.scale(training_set)
inputs_5 = get_input(inputs_5.tolist())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  # Remove the CWD from sys.path while we load stuff.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  if sys.path[0] == '':
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  app.launch_new_instance()
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.


In [10]:
print(len(inputs_4))

1523


In [11]:
print(len(LSTM_training_outputs))

1523


In [12]:
tinputs_1 = preprocessing.scale(test_set.drop("difficulty",1))
tinputs_1 = get_input(tinputs_1.tolist())
tinputs_2 = preprocessing.scale(test_set.drop("price",1))
tinputs_2 = get_input(tinputs_2.tolist())
tinputs_3 = preprocessing.scale(test_set.drop("recipients",1))
tinputs_3 = get_input(tinputs_3.tolist())
tinputs_4 = preprocessing.scale(test_set.drop("num_tranc",1))
tinputs_4 = get_input(tinputs_4.tolist())
tinputs_5 = preprocessing.scale(test_set)
tinputs_5 = get_input(tinputs_5.tolist())

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You m

In [13]:

from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout

def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()

    model.add(LSTM(neurons, input_shape=(inputs.shape[1], inputs.shape[2])))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

Using TensorFlow backend.


In [20]:
np.random.seed(203)
# eth_model_1 = build_model(inputs_1, output_size=1, neurons = 20)
# eth1_history = eth_model_1.fit(inputs_1, LSTM_training_outputs, 
#                             epochs=50, batch_size=1,verbose=2, shuffle=True)
# eth_model_2 = build_model(inputs_2, output_size=1, neurons = 20)
# eth2_history = eth_model_2.fit(inputs_2, LSTM_training_outputs, 
#                             epochs=50, batch_size=1,verbose=2, shuffle=True)
# eth_model_3 = build_model(inputs_3, output_size=1, neurons = 20)
# eth3_history = eth_model_3.fit(inputs_3, LSTM_training_outputs, 
#                             epochs=50, batch_size=1,verbose=2, shuffle=True)
eth_model_4 = build_model(inputs_4, output_size=1, neurons = 20)
eth4_history = eth_model_4.fit(inputs_4, LSTM_training_outputs, 
                            epochs=50, batch_size=1,verbose=2, shuffle=True)
eth_model_5 = build_model(inputs_5, output_size=1, neurons = 20)
eth5_history = eth_model_5.fit(inputs_5, LSTM_training_outputs, 
                            epochs=50, batch_size=1,verbose=20, shuffle=True)

Epoch 1/50
 - 25s - loss: 0.1747
Epoch 2/50
 - 23s - loss: 0.1194
Epoch 3/50
 - 24s - loss: 0.1030
Epoch 4/50
 - 23s - loss: 0.1036
Epoch 5/50
 - 23s - loss: 0.0982
Epoch 6/50
 - 22s - loss: 0.0933
Epoch 7/50
 - 22s - loss: 0.0920
Epoch 8/50
 - 21s - loss: 0.0889
Epoch 9/50
 - 22s - loss: 0.0930
Epoch 10/50
 - 22s - loss: 0.0916
Epoch 11/50
 - 22s - loss: 0.0906
Epoch 12/50
 - 22s - loss: 0.0893
Epoch 13/50
 - 23s - loss: 0.0846
Epoch 14/50
 - 22s - loss: 0.0835
Epoch 15/50
 - 23s - loss: 0.0839
Epoch 16/50
 - 23s - loss: 0.0844
Epoch 17/50
 - 23s - loss: 0.0869
Epoch 18/50
 - 22s - loss: 0.0828
Epoch 19/50
 - 21s - loss: 0.0857
Epoch 20/50
 - 23s - loss: 0.0798
Epoch 21/50
 - 23s - loss: 0.0837
Epoch 22/50
 - 23s - loss: 0.0804
Epoch 23/50
 - 23s - loss: 0.0843
Epoch 24/50
 - 23s - loss: 0.0841
Epoch 25/50
 - 23s - loss: 0.0824
Epoch 26/50
 - 23s - loss: 0.0838
Epoch 27/50
 - 23s - loss: 0.0829
Epoch 28/50
 - 22s - loss: 0.0809
Epoch 29/50
 - 22s - loss: 0.0803
Epoch 30/50
 - 23s - lo

In [21]:
# fig, ax1 = plt.subplots(1,1)

# ax1.plot(eth_history.epoch, eth_history.history['loss'])

# #ax1.set_title('Training Error')

# # if eth_model.loss == 'mae':
# #     ax1.set_ylabel('Mean Absolute Error (MAE)',fontsize=12)
# # # just in case you decided to change the model loss calculation
# # else:
# #     ax1.set_ylabel('Model Loss',fontsize=12)
# # ax1.set_xlabel('# Epochs',fontsize=12)
# plt.show()

LSTM_test_outputs = preprocessing.scale(test_set["price"])[10:]
output4 = eth_model_4.predict(tinputs_4)
# output1 = eth_model_1.predict(tinputs_1)
# output2 = eth_model_2.predict(tinputs_2)
# output3 = eth_model_3.predict(tinputs_3)

#output5 = eth_model.predict(tinputs_5)

In [22]:
def get_rmse(test_predict):
    return np.sqrt(((LSTM_test_outputs - test_predict) ** 2).mean())

In [23]:
rmse1 = get_rmse(output1)
print(rmse1)
rmse2 = get_rmse(output2)
print(rmse2)
rmse3 = get_rmse(output3)
print(rmse3)
rmse4 = get_rmse(output4)
print(rmse4)
# rmse1 = get_rmse(output1)
# print(rmse1)
# 1 remove difficulty
# 2 remove price
# 3 remove recipients
# 4 remove num_tranc

1.3903702761835828
1.1678219975599458
1.358654603396754
1.311267787185434


In [55]:
eth_model_5 = build_model(inputs_5, output_size=1, neurons = 20)
eth5_history = eth_model_5.fit(inputs_5, LSTM_training_outputs, 
                            epochs=55, batch_size=1,verbose=2, shuffle=True)

Epoch 1/55
 - 23s - loss: 0.1814
Epoch 2/55
 - 21s - loss: 0.1159
Epoch 3/55
 - 21s - loss: 0.1081
Epoch 4/55
 - 21s - loss: 0.0997
Epoch 5/55
 - 21s - loss: 0.0953
Epoch 6/55
 - 21s - loss: 0.0950
Epoch 7/55
 - 21s - loss: 0.0919
Epoch 8/55
 - 21s - loss: 0.0881
Epoch 9/55
 - 21s - loss: 0.0883
Epoch 10/55
 - 21s - loss: 0.0907
Epoch 11/55
 - 24s - loss: 0.0869
Epoch 12/55
 - 24s - loss: 0.0874
Epoch 13/55
 - 24s - loss: 0.0842
Epoch 14/55
 - 23s - loss: 0.0851
Epoch 15/55
 - 23s - loss: 0.0897
Epoch 16/55
 - 23s - loss: 0.0843
Epoch 17/55
 - 22s - loss: 0.0840
Epoch 18/55
 - 22s - loss: 0.0857
Epoch 19/55
 - 22s - loss: 0.0842
Epoch 20/55
 - 22s - loss: 0.0830
Epoch 21/55
 - 22s - loss: 0.0790
Epoch 22/55
 - 22s - loss: 0.0809
Epoch 23/55
 - 24s - loss: 0.0809
Epoch 24/55
 - 22s - loss: 0.0823
Epoch 25/55
 - 22s - loss: 0.0764
Epoch 26/55
 - 21s - loss: 0.0817
Epoch 27/55
 - 21s - loss: 0.0830
Epoch 28/55
 - 22s - loss: 0.0806
Epoch 29/55
 - 22s - loss: 0.0822
Epoch 30/55
 - 22s - lo

In [73]:
output5 = eth_model_5.predict(tinputs_5)

In [75]:
rmse5 = get_rmse(output5)
print(rmse5)

1.316610956648462
